In [1]:
from collections import defaultdict
import statistics
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

/work/nashokkumar_umass_edu/.conda/envs/nistorch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [3]:
max_source_length = 512
max_target_length = 128

In [4]:
# Define two training examples
input_sequence_1 = "Welcome to NYC"
output_sequence_1 = "Bienvenue à NYC"

input_sequence_2 = "HuggingFace is a company"
output_sequence_2 = "HuggingFace est une entreprise"

In [5]:
task_prefix = "translate English to French: "
input_sequences = [input_sequence_1, input_sequence_2]

In [6]:
# Get input encoding
encoding = tokenizer(
    [task_prefix + sequence for sequence in input_sequences],
    padding="longest",
    max_length=max_source_length,
    truncation=True,
    return_tensors="pt",
)

In [7]:
input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

In [8]:
print(input_ids)
print(attention_mask)

tensor([[13959,  1566,    12,  2379,    10,  5242,    12, 13465,     1,     0,
             0,     0,     0,     0],
        [13959,  1566,    12,  2379,    10, 11560,  3896,   371,  3302,    19,
             3,     9,   349,     1]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [9]:
target_encoding = tokenizer(
    [output_sequence_1, output_sequence_2],
    padding="longest",
    max_length=max_target_length,
    truncation=True,
    return_tensors="pt",
)

labels = target_encoding.input_ids

In [10]:
print(labels)

tensor([[10520, 15098,     3,    85, 13465,     1,     0,     0],
        [11560,  3896,   371,  3302,   259,   245, 11089,     1]])


In [11]:
print(tokenizer.pad_token_id) # pad_token_id

0


In [12]:
labels[labels == tokenizer.pad_token_id] = -100

In [13]:
loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
loss.item()

0.18801377713680267

In [13]:
# load dataset
def get_parallel_corpus(ip_df, story_df):
    # hash stories and sections
    story_sec_hash = defaultdict(dict)
    for i, row in story_df.iterrows():
        story_sec_hash[row['source_title']][row['cor_section']] = row['text']
    
    story, answer, question = [], [], []
    for i, row in ip_df.iterrows():
        sec_nums = row['cor_section'].split(',')
        story_str = ''
        for sec_num in sec_nums:
            story_str += story_sec_hash[row['source_title']][int(sec_num)]
        story.append(story_str)
        answer.append(row['answer'])
        question.append(row['question'])
    
    return story, answer, question

In [14]:
story_file = '../../data/original/source_texts.csv'
story_df = pd.read_csv(story_file)
# Train-Val split
train_file = '../../data/train_val_split_csv/train.csv'
train_df = pd.read_csv(train_file)
val_file = '../../data/train_val_split_csv/val.csv'
val_df = pd.read_csv(val_file)

train_story, train_answer, train_question = get_parallel_corpus(train_df, story_df)
val_story, val_answer, val_question = get_parallel_corpus(val_df, story_df)

In [15]:
def get_stats(story, answer, question):
    print('Average story length:', statistics.mean([len(stry) for stry in story]))
    print('Average answer length:', statistics.mean([len(ans) for ans in answer]))
    print('Average question length:', statistics.mean([len(quest) for quest in question]))

In [16]:
# print stats
print('Train Set')
get_stats(train_story, train_answer, train_question)

print('Valid Set')
get_stats(val_story, val_answer, val_question)

Train Set
Average story length: 1015.8474604496253
Average answer length: 36.77901748542881
Average question length: 51.96319733555371
Valid Set
Average story length: 987.5924796747968
Average answer length: 33.672764227642276
Average question length: 48.9369918699187
